<a href="https://colab.research.google.com/github/akshaya-code/Assignment1/blob/master/actor_critic_dac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
class VLSICell:
    def __init__(self, node_id, lower_left_x, lower_left_y, width, height, move_type):
        self.node_id = node_id
        self.lower_left_x = lower_left_x
        self.lower_left_y = lower_left_y
        self.width = width
        self.height = height
        self.move_type = move_type
        self.mid_x = None
        self.mid_y = None
        self.pin_offsets = []
        self.update_midpoint()

    def update_midpoint(self):
        if self.width is not None and self.height is not None:
            self.mid_x = self.lower_left_x + self.width / 2
            self.mid_y = self.lower_left_y + self.height / 2

    def update_position(self, lower_left_x, lower_left_y):
        self.lower_left_x = lower_left_x
        self.lower_left_y = lower_left_y
        self.update_midpoint()

    def set_dimensions(self, width, height):
        self.width = width
        self.height = height
        self.update_midpoint()

    def add_pin_offset(self, offset_x, offset_y):
        self.pin_offsets.append((offset_x, offset_y))

    def __repr__(self):
        return (f"VLSICell(node_id='{self.node_id}', x={self.lower_left_x}, y={self.lower_left_y}, "
                f"width={self.width}, height={self.height}, mid_x={self.mid_x}, mid_y={self.mid_y}, "
                f"move_type='{self.move_type}', pin_offsets={self.pin_offsets})")

def parse_vlsi_file(file_path):
    cells = []
    with open(file_path, 'r') as file:
        for _ in range(4):
            next(file)
        for line in file:
            parts = line.split()
            if len(parts) < 4:
                continue
            node_id = parts[0]
            lower_left_x = int(parts[1])
            lower_left_y = int(parts[2])
            move_type = parts[5].lstrip("/") if len(parts) >= 6 else "movable"
            cell = VLSICell(node_id, lower_left_x, lower_left_y, None, None, move_type)
            cells.append(cell)
    return cells

def parse_width_height_file(file_path):
    dimensions = {}
    with open(file_path, 'r') as file:
        for _ in range(7):
            next(file)
        for line in file:
            parts = line.split()
            if len(parts) < 3:
                continue
            node_id = parts[0]
            width = float(parts[1])
            height = float(parts[2])
            dimensions[node_id] = (width, height)
    return dimensions

def parse_net_file(file_path):
    net_data = {}
    with open(file_path, 'r') as file:
        for _ in range(7):
            next(file)
        current_net = None
        for line in file:
            line = line.strip()
            if line.startswith("NetDegree"):
                parts = line.split()
                current_net = parts[-1]
                net_data[current_net] = []
            elif current_net and line:
                parts = line.split(":")
                if len(parts) == 2:
                    node_name = parts[0].split()[0]
                    x_offset, y_offset = map(float, parts[1].split())
                    net_data[current_net].append({
                        "node_name": node_name,
                        "x_offset": x_offset,
                        "y_offset": y_offset
                    })
    return net_data

def integrate_all(file_pl_path, file_nodes_path, file_nets_path):
    cells = parse_vlsi_file(file_pl_path)
    cell_dict = {cell.node_id: cell for cell in cells}
    dimensions = parse_width_height_file(file_nodes_path)
    for node_id, (width, height) in dimensions.items():
        if node_id in cell_dict:
            cell_dict[node_id].set_dimensions(width, height)
    net_data = parse_net_file(file_nets_path)
    for pins in net_data.values():
        for pin in pins:
            node_id = pin["node_name"]
            if node_id in cell_dict:
                cell_dict[node_id].add_pin_offset(pin["x_offset"], pin["y_offset"])
    return cell_dict

# Example usage:
file_pl_path = "/content/superblue14.pl"
file_nodes_path = "/content/superblue14.nodes"
file_nets_path = "/content/superblue14.nets"
cells = integrate_all(file_pl_path, file_nodes_path, file_nets_path)

# Print a few sample cells
for cell in list(cells.values())[:10]:
    print(cell)


VLSICell(node_id='o0', x=0, y=0, width=8.0, height=9.0, mid_x=4.0, mid_y=4.5, move_type='movable', pin_offsets=[(-2.0, -0.5), (-3.0, 0.5), (3.0, -1.5), (2.0, -0.5), (0.0, -2.5)])
VLSICell(node_id='o1', x=0, y=0, width=5.0, height=9.0, mid_x=2.5, mid_y=4.5, move_type='movable', pin_offsets=[(-1.5, -0.5), (1.5, -0.5), (1.5, -1.5)])
VLSICell(node_id='o2', x=0, y=0, width=5.0, height=9.0, mid_x=2.5, mid_y=4.5, move_type='movable', pin_offsets=[(-1.5, 0.5), (-1.5, -0.5), (0.5, 0.5), (1.5, -1.5)])
VLSICell(node_id='o3', x=0, y=0, width=9.0, height=9.0, mid_x=4.5, mid_y=4.5, move_type='movable', pin_offsets=[(-3.5, 0.5), (-1.5, 0.5), (-2.5, 1.5), (3.5, -0.5)])
VLSICell(node_id='o4', x=0, y=0, width=4.0, height=9.0, mid_x=2.0, mid_y=4.5, move_type='movable', pin_offsets=[(0.0, -2.5), (1.0, -1.5), (1.0, 1.5), (0.0, 3.5)])
VLSICell(node_id='o5', x=0, y=0, width=30.0, height=9.0, mid_x=15.0, mid_y=4.5, move_type='movable', pin_offsets=[(8.0, -0.5), (1.0, -0.5), (-4.0, -0.5), (-13.0, -3.5)])
VLSIC

In [3]:
class Row:
    def __init__(self, subrow_origin, num_sites, site_spacing, height, y_coordinate):
        self.subrow_origin = subrow_origin
        self.num_sites = num_sites
        self.site_spacing = site_spacing
        self.height = height
        self.y_coordinate = y_coordinate

    def compute_width(self):
        return self.num_sites * self.site_spacing

    def compute_area(self):
        return self.compute_width() * self.height

    def __repr__(self):
        return (f"Row(subrow_origin={self.subrow_origin}, "
                f"num_sites={self.num_sites}, site_spacing={self.site_spacing}, "
                f"height={self.height}, y_coordinate={self.y_coordinate})")  # Include y_coordinate


def parse_chip_file(file_path):
    rows = []
    total_area = 0
    x_min = float('inf')
    x_max = float('-inf')
    y_min = float('inf')
    y_max = float('-inf')
    height_per_row = None

    try:
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()

                if not line or line.startswith("#"):
                    continue

                if line.startswith("CoreRow Horizontal"):
                    subrow_origin = None
                    num_sites = None
                    site_spacing = None
                    height = None
                    row_y = None

                    while True:
                        line = file.readline().strip()
                        if not line or line.startswith("#"):
                            continue
                        if line.startswith("End"):
                            break

                        if "Coordinate" in line:
                            parts = line.split(":")
                            row_y = float(parts[1].strip())
                            y_min = min(y_min, row_y)
                        elif "Height" in line:
                            height = float(line.split(":")[1].strip())
                            height_per_row = height
                        elif "Sitespacing" in line:
                            site_spacing = float(line.split(":")[1].strip())
                        elif "SubrowOrigin" in line and "NumSites" in line:
                            parts = line.split(":")
                            subrow_origin = float(parts[1].split()[0])
                            num_sites = int(parts[2].strip())

                    if all([subrow_origin, num_sites, site_spacing, height, row_y]):  # Check all parameters
                        row = Row(subrow_origin, num_sites, site_spacing, height, row_y)
                        rows.append(row)
                        total_area += row.compute_area()

                        x_min = min(x_min, subrow_origin)
                        x_max = max(x_max, subrow_origin + row.compute_width())
                        y_max = max(y_max, row_y + height)

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return [], None, None
    except Exception as e:
        print(f"Error while parsing file: {e}")
        return [], None, None

    chip_width = x_max - x_min
    chip_height = y_max - y_min

    return rows, total_area, (x_min, y_min, chip_width, chip_height)
# Example usage
if __name__ == "__main__":
    file_path = "/content/superblue14.scl"
    rows, chip_area, chip_dimensions = parse_chip_file(file_path)
    print(f"Total chip area: {chip_area}")
    print(f"Chip dimensions (x_min, y_min, width, height): {chip_dimensions}")
    print("First 5 rows:")
    for row in rows[:5]:
        print(row)


Total chip area: 244168776.0
Chip dimensions (x_min, y_min, width, height): (18.0, 18.0, 12956.0, 18846.0)
First 5 rows:
Row(subrow_origin=18.0, num_sites=12956, site_spacing=1.0, height=9.0, y_coordinate=18.0)
Row(subrow_origin=18.0, num_sites=12956, site_spacing=1.0, height=9.0, y_coordinate=27.0)
Row(subrow_origin=18.0, num_sites=12956, site_spacing=1.0, height=9.0, y_coordinate=36.0)
Row(subrow_origin=18.0, num_sites=12956, site_spacing=1.0, height=9.0, y_coordinate=45.0)
Row(subrow_origin=18.0, num_sites=12956, site_spacing=1.0, height=9.0, y_coordinate=54.0)


In [4]:

import torch
import torch.nn as nn
import torch.optim as optim
import random

class Actor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Actor, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Softmax(dim=-1)
        )

    def forward(self, x):
        return self.fc(x)

class Critic(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Critic, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        return self.fc(x)

class ActorCriticAgent:
    def __init__(self, env, input_dim=3, hidden_dim=128):
        self.env = env
        self.actor = Actor(input_dim, hidden_dim, len(env.rows))
        self.critic = Critic(input_dim, hidden_dim)
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=0.001)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=0.005)
        self.gamma = 0.99

    def get_state_tensor(self, cell):
        # Normalize features: width, height, pin count
        return torch.tensor([
            cell.width / self.env.width,
            cell.height / self.env.height,
            len(cell.pin_offsets) / 50.0
        ], dtype=torch.float32)

    def select_action(self, cell):
        state = self.get_state_tensor(cell)
        probs = self.actor(state)
        row_index = torch.multinomial(probs, 1).item()
        y = self.env.rows[row_index].y_coordinate
        x = random.randint(0, int(self.env.width - cell.width))
        return (cell.node_id, x, y), state, row_index

    def update(self, state, reward, next_state, row_index):
        value = self.critic(state)
        next_value = self.critic(next_state).detach()
        advantage = reward + self.gamma * next_value - value

        critic_loss = advantage.pow(2).mean()
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        log_prob = torch.log(self.actor(state)[row_index])
        actor_loss = -log_prob * advantage.detach()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()


In [5]:

import math
from typing import List, Tuple

class PlacementEnv:
    def __init__(self, cells: List, rows: List, width: float, height: float):
        self.cells = cells
        self.rows = rows
        self.width = width
        self.height = height
        self.placed_cells = set()
        self.current_index = 0
        self.done = False

    def is_legal(self, cell, x, y):
        for row in self.rows:
            if math.isclose(row.y_coordinate, y, abs_tol=1e-3) and                row.subrow_origin <= x <= (row.subrow_origin + row.compute_width() - cell.width):
                return True
        return False

    def compute_hpwl(self):
        net_positions = {}
        for cell in self.cells:
            for offset_x, offset_y in cell.pin_offsets:
                pin_x = cell.mid_x + offset_x
                pin_y = cell.mid_y + offset_y
                net_positions.setdefault(cell.node_id, []).append((pin_x, pin_y))

        hpwl = 0
        for pins in net_positions.values():
            xs, ys = zip(*pins)
            hpwl += (max(xs) - min(xs)) + (max(ys) - min(ys))
        return hpwl

    def step(self, action: Tuple[str, float, float]):
        if self.done:
            return None, 0, True, {}

        node_id, x, y = action
        cell = next((c for c in self.cells if c.node_id == node_id), None)

        if not cell:
            return None, -10, False, {"error": "Invalid cell ID"}

        if not self.is_legal(cell, x, y):
            return None, -5, False, {"error": "Illegal placement"}

        cell.update_position(x, y)
        self.placed_cells.add(cell.node_id)
        reward = -self.compute_hpwl()

        self.current_index += 1
        if self.current_index >= len(self.cells):
            self.done = True

        return (cell.mid_x, cell.mid_y), reward, self.done, {}

    def reset(self):
        self.placed_cells = set()
        self.current_index = 0
        self.done = False
        for cell in self.cells:
            cell.update_position(0, 0)
        return self.cells[0].mid_x, self.cells[0].mid_y

    def render(self):
        return {
            cell.node_id: (cell.lower_left_x, cell.lower_left_y)
            for cell in self.cells
        }


In [ ]:

import torch

file_pl = "/content/superblue14.pl"
file_nodes = "/content/superblue14.nodes"
file_nets = "/content/superblue14.nets"
file_scl = "/content/superblue14.scl"

cells_dict = integrate_all(file_pl, file_nodes, file_nets)
rows, _, (x_min, y_min, width, height) = parse_chip_file(file_scl)
cells = list(cells_dict.values())

env = PlacementEnv(cells, rows, width, height)
agent = ActorCriticAgent(env)

episodes = 3
for ep in range(1, episodes + 1):
    print(f"\n--- Episode {ep}/{episodes} ---")
    env.reset()
    total_reward = 0
    for i, cell in enumerate(cells):
        action, state, row_index = agent.select_action(cell)
        _, reward, done, _ = env.step(action)
        next_state = agent.get_state_tensor(cell)
        agent.update(state, reward, next_state, row_index)
        total_reward += reward
        if i % 100 == 0:
            print(f"Placed {i}/{len(cells)} cells...", end="\r")
        if done:
            break
    print(f"Episode {ep} completed. Total Reward: {total_reward}, Final HPWL: {-total_reward}")



--- Episode 1/3 ---


In [ ]:
import matplotlib.pyplot as plt

def plot_rewards(rewards, title="Total Reward per Episode"):
    plt.figure(figsize=(10, 5))
    plt.plot(rewards)
    plt.title(title)
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.grid(True)
    plt.show()
